In [1]:
!pip install chromadb 

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.9-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached onnxruntime-1.21.1-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.32.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentel

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.


In [2]:
from langchain.chains import RetrievalQA
import wikipedia
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.utilities import OpenWeatherMapAPIWrapper,WikipediaAPIWrapper


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [4]:
llm = OpenAI(temperature=0)

In [5]:
#OpenWeatherMap
#weather_api = OpenWeatherMapAPIWrapper()  #https://home.openweathermap.org/api_keys

wiki = wikipedia #WikipediaAPIWrapper()



In [13]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI  # لو عندك OpenAI أو أي موديل آخر

# 1. تحليل السؤال لاستخراج الموضوع المناسب
def extract_topic(user_input):
    """استخدم موديل LLM بسيط لتحليل النص واستخراج الكلمة المفتاحية."""
 #   llm = OpenAI(temperature=0.3, model_name="gpt-3.5-turbo")
    
    prompt = f"من النص التالي، استخرج الكلمة أو الموضوع الذي أبحث عنه للبحث في ويكيبيديا. لا تشرح، فقط أعطني الكلمة أو الجملة المناسبة:\n\n{user_input}"
    topic = llm.invoke(prompt)
    return topic.strip()

# 2. بحث في ويكيبيديا بناءً على الموضوع المستخرج
def search_wikipedia(user_input):
    #wiki = WikipediaAPIWrapper(top_k_results=3, lang='en')
    
    topic = extract_topic(user_input)  # هنا نحصل على الكلمة أو الفكرة
    print(f"البحث عن: {topic}")
    
    result = wiki.search(topic)
    
    document = Document(page_content=result, metadata={"topic": topic})
    
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents([document])
    
    return texts

# 📥 مثال استخدام:
user_question = "خطط لي رحلة لعالم ديزني"
texts = search_wikipedia(user_question)

# عرض النتائج:
for i, chunk in enumerate(texts):
    print(f"جزء {i+1} - الموضوع: {chunk.metadata['topic']}")
    print(chunk.page_content)
    print("-" * 80)


البحث عن: عالم ديزني


ValidationError: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type

In [ ]:
# loader = TextLoader(wiki)
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()
# docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

RuntimeError: Error loading wiki_client=<module 'wikipedia' from 'c:\\Users\\riina\\anaconda3\\Lib\\site-packages\\wikipedia\\__init__.py'> top_k_results=3 lang='en' load_all_available_meta=False doc_content_chars_max=4000